# Guest Session Feature Demo

This notebook is a demo of a feature that WhyLabs is working on called Guest Sessions. The feature builds ontop of our open source library, whylogs, to make it easier to visualize datasets and identify data issues. 

Typically when using whylogs, you'll log data to using one of the `whylogs.log()` apis to generate JSON and/or binary profiles locally that summarize various statistics about the dataset. That feature isn't going anywhere, don't worry! Guest Sessions take those generated statistics and uploads them to WhyLabs.ai to populate our dashboard without the need to sign in or share any personal identifiable information.

## How it works

The rest of the notebook will go through how the feature works and how to use it, starting with imports.

In [1]:
import pandas as pd
from datetime import datetime, timedelta
from whylogs.app.session import start_whylabs_session
from tqdm.auto import tqdm

### Load the data

Next, we'll load a sample dataset for logging with whylogs. The dataset was based on a set [from Kaggle](https://www.kaggle.com/yugagrawal95/sample-media-spends-data) for various advertising campaigns. We'll just use the subset of data from April months.

In [2]:
csv_file = "data/sample_media_spend.csv"
csv_dataframe = pd.read_csv(csv_file, parse_dates=["Calendar_Week"])
csv_dataframe

,Division,Calendar_Week,Paid_Views,Organic_Views,Google_Impressions,Email_Impressions,Facebook_Impressions,Affiliate_Impressions,Overall_Views,Sales
0,A,2018-01-06,392,422,408,3.498950e+05,73580,12072,682,59417
1,A,2018-01-13,787,904,110,5.062702e+05,11804,9499,853,56806
2,A,2018-01-20,81,970,742,4.300422e+05,52232,17048,759,48715
3,A,2018-01-27,25,575,65,4.177457e+05,78640,10207,942,72047
4,A,2018-02-03,565,284,295,4.085058e+05,40561,5834,658,56235
...,...,...,...,...,...,...,...,...,...,...
3046,Z,2020-02-01,29239,25311,622406,1.459071e+06,45026,12098,53667,82707
3047,Z,2020-02-08,26230,28031,624409,5.342505e+05,227070,9548,53665,84503
3048,Z,2020-02-15,24749,31281,439362,4.227182e+05,393685,9861,55561,147325
3049,Z,2020-02-22,20713,30356,464178,6.085799e+05,424676,10221,49221,111525


### Group the data

We'll split the data into training and test data with a somewhat arbitrary date as the border.

In [3]:
model_date = datetime(2020, 1, 1)
training_data = csv_dataframe[csv_dataframe["Calendar_Week"] < model_date]
test_data = csv_dataframe[csv_dataframe["Calendar_Week"] >= model_date]

### Create a Guest Session

Next, we get to the main feature. We'll start a Guest Session with the `start_whylabs_session` api. The important thing about this API to notice is that there is a consent flag to data uploading.

```python
start_whylabs_session(data_collection_consent=True)
```

We just want to make it clear that this does upload statistical data to WhyLabs.ai. We don't ever upload raw data, only the profiles that we generate via logging. The flag has to be `True` or the feature will bail out.

In [4]:
session = start_whylabs_session(data_collection_consent=True, report_progress=True)

WARN: Missing config


### Log the data

Next we'll actually log the data and generate the statistical profiles. This will generate a profile for each day in the data that we're logging.

In [5]:
data = [
    [training_data, "training", datetime.now()], 
    [test_data, "test", datetime.now() - timedelta(days=1)]
]

for df, name, ts in tqdm(data):
    logger = session.logger(dataset_name=name, dataset_timestamp=ts).log_dataframe(df)

  0%|          | 0/2 [00:00<?, ?it/s]

### Close the session and upload the generated profiles

Finally, we close the session which causes the profile uploads to WhyLabs.ai. The progress bar reflects the upload process. When it's done it will print out a URL that you can use to view the data in the WhyLabs.ai dashboard.

In [6]:
with session:
    print("done")

done


  0%|          | 0/2 [00:00<?, ?it/s]

You can explore your data in Observatory here: https://hub.whylabsapp.com/models/model-1/profiles/?sessionToken=session-12678


### View your data in WhyLabs

Clicking the URL above will bring you to the WhyLabs dashboard. You'll see a banner at the top that indicates that we may still be processing data in our backend. You'll land on our profile viewer page and you'll be able to see the uploaded profiles from here, possibly before all of the data finishes processing.